In [79]:
import os, sys
import pandas as pd
import numpy as np
from functools import reduce

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from Scripts import Data_Loader_Functions as dL
from Scripts import Results_Evaluation as rE
RESULTS = os.path.join(module_path, 'Results', 'Paper')
DATA = os.path.join(module_path, 'Data', 'Augmented Data')

In [102]:
# Create DF with TP TN FP FN
model_name = 'PFDL'
df = pd.read_csv(os.path.join(RESULTS, '20200223 - ' + model_name + "_all_final.csv"))

In [103]:
# Load Pivot & Subjects
pivot = dL.create_pivot(os.path.join(DATA, 'group_2'), 'Session', 'Person', 'Session')
subjects = dL.create_pain_df(os.path.join(DATA, 'group_2'))['Person'].unique()

In [104]:
def filter_granular_results(df, subjects, pivot, model_name):
    
    # Add columns to df [TP, TN, FP, FN]
    df['label'] = np.minimum(df['True Label'], 1)
    df['prediction'] = df[model_name].round()
    df['TP'] = (df['label'] == 1) & (df['prediction'] == 1)
    df['TN'] = (df['label'] == 0) & (df['prediction'] == 0)
    df['FP'] = (df['label'] == 0) & (df['prediction'] == 1)
    df['FN'] = (df['label'] == 1) & (df['prediction'] == 0)

    # Prepare pivot to be merged with df
    columns = np.insert(subjects.astype(object), 0, 'Session')
    pivot = pivot.reset_index()[columns].drop(0).reset_index(drop=True)
    pivot_has_positive = pivot[pivot.columns[1:]] != ''
    pivot_has_positive['Session'] = pivot['Session']
    pivot_has_positive = pivot_has_positive.drop(9)
    unpivot_has_positive = pivot_has_positive.melt(id_vars=['Session'])
    unpivot_has_positive = unpivot_has_positive.rename(columns={'Person': 'Subject ID', 'value': 'has_positive'})

    # Merge pivot and df to indicate, which subjects have positive examples in a given session
    df_merge = df.merge(unpivot_has_positive)
    
    # Return only those session/subject combinations that have positive examples
    return df_merge[df_merge['has_positive']]

In [105]:
df_merge_has_positive = filter_granular_results(df, subjects, pivot, model_name)

In [106]:
df_merge_has_positive.to_csv(os.path.join(RESULTS, '20200223 - ' + model_name + "_all_final_positive.csv"))